# Scripts Mosaicos

In [1]:
import ee
import geemap
import os
import folium

In [2]:
Map = geemap.Map()
Map2 = geemap.Map()
Map3 = geemap.Map()

## Área de Mosaicos

### Patagonia

In [3]:
work_dir = os.path.expanduser('../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)
Map.addLayer(patagonia_grande, {}, 'Patagonia')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## Funciones

In [4]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [5]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B3','B5']).rename('NDWI')
    return image.addBands(ndwi)

In [6]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B3','B4']).rename('NDGI')
    return image.addBands(ndgi)

In [7]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [40]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [9]:
def Cloud50(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(50))  

In [10]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [11]:
def Agua(image):
    return image.updateMask(ndwi.lte(-0.1))

In [12]:
def Agua2021(image):
    return image.updateMask(ndwi2021.gte(0.1))

In [13]:
def Agua2020(image):
    return image.updateMask(ndwi2020.gte(0))

In [14]:
def Agua2019(image):
    return image.updateMask(ndwi2019.gte(0))

In [15]:
def Ndvi2021(image):
    return image.updateMask(ndvi2021.gt(0.089))

In [76]:
def Ndvi2020(image):
    return image.updateMask(ndvi2020.gt(0.089))

In [17]:
def Ndvi2019(image):
    return image.updateMask(ndvi2019.lt(0.2))

In [18]:
def Ndvi2(image):
    return image.updateMask(ndvi.gte(0.2))

## Colecciones

### Verano 2021

In [116]:
start_date = '2020-10-01'
end_date = '2021-03-31'

l8_2021 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [117]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [118]:
coleccion2021 = l8_2021.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

### Verano 2020

In [119]:
start_date = '2019-10-01'
end_date = '2020-03-31'

l8_2020 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [120]:
coleccion2020 = l8_2020.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

### Verano 2019

In [121]:
start_date = '2018-10-01'
end_date = '2019-03-31'

l8_2019 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [122]:
coleccion2019 = l8_2019.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

## Quality Mosaic NDVI

In [26]:
NDVI_2021 = coleccion2021.qualityMosaic('NDVI')
NDVI_2020 = coleccion2020.qualityMosaic('NDVI')
NDVI_2019 = coleccion2019.qualityMosaic('NDVI')

In [27]:
visParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map.addLayer(NDVI_2019, visParams, 'Greenest 2019')
Map.addLayer(NDVI_2020, visParams, 'Greenest 2020')
Map.addLayer(NDVI_2021, visParams, 'Greenest 2021')
Map.centerObject(patagonia_grande, 6)
Map

Map(bottom=1851.0, center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position…

In [28]:
ndvi2021 = NDVI_2021.select('NDVI')
ndvi2020 = NDVI_2020.select('NDVI')
ndvi2019 = NDVI_2019.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']

In [29]:
Map.addLayer(ndvi2021, {'palette': palette}, 'NDVI_2021')
Map.addLayer(ndvi2020, {'palette': palette}, 'NDVI_2020')
Map.addLayer(ndvi2019, {'palette': palette}, 'NDVI_2019')
Map

Map(bottom=1851.0, center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position…

## Agua

### Verano 2021

In [85]:
aquast_2021 = coleccion2021.qualityMosaic('NDWI')

### Verano 2020

In [86]:
aquast_2020 = coleccion2020.qualityMosaic('NDWI')

### Verano 2019

In [87]:
aquast_2019 = coleccion2019.qualityMosaic('NDWI')

### Quality Mosaic NDWI

In [88]:
visParams = {
  'bands': ['B4', 'B3', 'B1'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map2.addLayer(aquast_2019, visParams, 'Agua 2019')
Map2.addLayer(aquast_2020, visParams, 'Agua 2020')
Map2.addLayer(aquast_2021, visParams, 'Agua 2021')
Map2.centerObject(patagonia_grande, 6)
Map2

Map(bottom=10981.0, center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['positio…

In [89]:
ndwi2021 = aquast_2021.select('NDWI')
ndwi_viz = ['00FFFF', '0000FF']
ndwi2020 = aquast_2020.select('NDWI')
ndwi2019 = aquast_2019.select('NDWI')
Map2.addLayer(ndwi2021, {'palette': ndwi_viz}, 'NDWI_2021')
Map2.addLayer(ndwi2020, {'palette': ndwi_viz}, 'NDWI_2020')
Map2.addLayer(ndwi2019, {'palette': ndwi_viz}, 'NDWI_2019')
Map2

Map(bottom=10981.0, center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['positio…

In [90]:
ndwi_rgb = ndwi2021.visualize(**{
    'min': 0.5,
    'max':1,
    'palette': ['00FFFF', '0000FF']
})

In [91]:
ndwi_mask = ndwi2021.updateMask(ndwi2021.lte(0.1))

In [92]:
map_ndwi_mask = folium.Map(location = [-45, -73], zoom_start = 5)
map_ndwi_mask.add_ee_layer(ndwi_mask, ndwi_viz, 'NDWI sin tierra')
display(map_ndwi_mask)

TypeError: list indices must be integers or slices, not str

In [93]:
ndwi_mask2020 = ndwi2020.updateMask(ndwi2020.gte(0.1))

In [94]:
map_ndwi_mask2020 = folium.Map(location = [-45, -73], zoom_start = 5)
map_ndwi_mask2020.add_ee_layer(ndwi_mask2020, ndwi_viz, 'NDWI sin tierra')
display(map_ndwi_mask2020)

TypeError: list indices must be integers or slices, not str

## Máscaras

### Verano 2021

#### Nubes 50

In [45]:
coleccion21 = coleccion2021.map(Cloud50)
greenest21 = coleccion21.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2021.add_ee_layer(greenest21, vizParams, 'Mapa sin Nubes')
display(map_2021)

#### Nubes 30

In [46]:
coleccion21 = coleccion2021.map(Cloud30)
greenest21 = coleccion21.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2021.add_ee_layer(greenest21, vizParams, 'Mapa sin Nubes')
display(map_2021)

#### Filtro de agua

In [47]:
coleccion21_ = coleccion2021.map(Cloud50)
coleccion21_ = coleccion21_.map(Agua2021)
greenest21_ = coleccion21_.qualityMosaic('NDVI')
map_2021_ = folium.Map(location = [-45, -73], zoom_start = 5)
map_2021_.add_ee_layer(greenest21_, vizParams, 'Mapa sin Nubes')
display(map_2021_)

In [48]:
agua_2021 = coleccion2021.map(Cloud50)
agua_2021 = agua_2021.map(Agua2021)
aquast21_ = agua_2021.qualityMosaic('NDWI')
map_agua2021 = folium.Map(location = [-45, -73], zoom_start = 5)
map_agua2021.add_ee_layer(aquast21_, vizParams, 'Mapa sin Nubes')
display(map_agua2021)

In [49]:
coleccion21 = coleccion2021.map(Cloud50)
greenest21 = coleccion21.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2021.add_ee_layer(greenest21, vizParams, 'Mapa sin Nubes')
display(map_2021)

#### Filtro NDVI

In [50]:
coleccion21 = coleccion2021.map(Cloud50)
coleccion21 = coleccion21.map(Ndvi2021)
greenest21 = coleccion21.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2021.add_ee_layer(greenest21, vizParams, 'Mapa sin Nubes')
display(map_2021)

### Verano 2020

In [123]:
coleccion20 = coleccion2020.map(Cloud30)
# coleccion20 = coleccion20.map(Ndvi2021)
greenest20 = coleccion20.qualityMosaic('NDVI')
map_2020 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2020.add_ee_layer(greenest20, vizParams, 'Mapa sin nubes')
display(map_2020)

In [52]:
agua_2020 = coleccion2020.map(Agua2020)
aquast20_ = agua_2020.qualityMosaic('NDWI')
map_agua2020 = folium.Map(location = [-45, -73], zoom_start = 5)
map_agua2020.add_ee_layer(aquast20_, vizParams, 'Mapa sin Nubes')
display(map_agua2020)

### Verano 2019

In [69]:
coleccion2019 = coleccion2019.map(Cloud30)
# coleccion2019 = coleccion2019.map(Agua)
greenest2019 = coleccion2021.qualityMosaic('NDVI')
map_2019 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2019.add_ee_layer(greenest2019, vizParams, 'Mapa sin nubes')
display(map_2019)

## Mosaicos

In [124]:
mosaic = ee.ImageCollection([greenest2019,greenest20]).mosaic()

In [125]:
Map3.addLayer(mosaic, vizParams, 'Mosaico')
Map3

Map(bottom=22470.0, center=[-53.50111704294317, -71.63635253906251], controls=(WidgetControl(options=['positio…

In [126]:
mosaic2 = ee.ImageCollection([mosaic, greenest21]).mosaic()

In [127]:
Map3.addLayer(mosaic2, vizParams, 'Mosaico2')
Map3

Map(bottom=22470.0, center=[-53.50111704294317, -71.63635253906251], controls=(WidgetControl(options=['positio…

In [132]:
mosaic3 = ee.ImageCollection([mosaic2, aquast20_]).mosaic()

In [133]:
Map3.addLayer(mosaic3, vizParams, 'Mosaico3')
Map3

Map(bottom=22470.0, center=[-53.50111704294317, -71.63635253906251], controls=(WidgetControl(options=['positio…

In [134]:
mosaic4 = ee.ImageCollection([mosaic3, greenest21]).mosaic()

In [135]:
Map3.addLayer(mosaic4, vizParams, 'Mosaico 4')
Map3

Map(bottom=22470.0, center=[-53.50111704294317, -71.63635253906251], controls=(WidgetControl(options=['positio…

In [ ]:
mosaic4 = ee.ImageCollection([greenest19,mosaic3]).mosaic()

In [ ]:
Map3.addLayer(mosaic4, vizParams, 'Mosaico4')
Map3